In [4]:
%pip install requests pandas matplotlib seaborn bs4 lxml html5lib geopy scikit-learn

  Using cached scikit_learn-1.7.2-cp312-cp312-macosx_12_0_arm64.whl.metadata (11 kB)
  Using cached scipy-1.16.3-cp312-cp312-macosx_14_0_arm64.whl.metadata (62 kB)
  Using cached joblib-1.5.2-py3-none-any.whl.metadata (5.6 kB)
  Using cached threadpoolctl-3.6.0-py3-none-any.whl.metadata (13 kB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 125.4/125.4 kB 2.4 MB/s eta 0:00:00a 0:00:01
Using cached scikit_learn-1.7.2-cp312-cp312-macosx_12_0_arm64.whl (8.6 MB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.7/40.7 kB 2.6 MB/s eta 0:00:00
Using cached joblib-1.5.2-py3-none-any.whl (308 kB)
Using cached scipy-1.16.3-cp312-cp312-macosx_14_0_arm64.whl (20.9 MB)
Using cached threadpoolctl-3.6.0-py3-none-any.whl (18 kB)

[notice] A new release of pip is available: 24.0 -> 25.3
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [21]:
import requests
import pandas as pd
from bs4 import BeautifulSoup

url = "https://fr.wikipedia.org/wiki/Liste_des_gares_desservies_par_TGV"

html = requests.get(url).content
soup = BeautifulSoup(html, "html.parser")

tables = soup.find_all("table", {"class": "wikitable"})

all_rows = []

for table in tables:
    headers = [th.text.strip() for th in table.find_all("th")]
    
    for row in table.find_all("tr")[1:]:
        cells = [td.text.strip() for td in row.find_all(["td", "th"])]
        if len(cells) == len(headers):
            all_rows.append(dict(zip(headers, cells)))

df = pd.DataFrame(all_rows)

print(df.head())


Empty DataFrame
Columns: []
Index: []


In [3]:
import pandas as pd
import requests
import io

url = "https://fr.wikipedia.org/wiki/Liste_des_gares_desservies_par_TGV"

# 1. On se fait passer pour un navigateur web pour ne pas être bloqué
headers = {
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36'
}

try:
    # On fait la requête avec les headers et sans vérification SSL
    response = requests.get(url, headers=headers, verify=False)
    
    # On vérifie si la page a bien été chargée (Code 200 = OK)
    if response.status_code == 200:
        print("Page récupérée avec succès !")
        
        # 2. On utilise io.StringIO pour que Pandas lise le texte comme un fichier
        # 3. 'match="Département"' force Pandas à ne garder que les tables qui contiennent ce mot
        dfs = pd.read_html(io.StringIO(response.text), match="Département")
        
        if len(dfs) > 0:
            df_gares = dfs[0] # On prend la première table trouvée
            print("-" * 30)
            print(f"Tableau trouvé avec {len(df_gares)} gares !")
            print(df_gares.head())
        else:
            print("Aucune table correspondante trouvée.")
            
    else:
        print(f"Erreur : Wikipédia a renvoyé le code {response.status_code}")

except Exception as e:
    print(f"Une erreur s'est produite : {e}")

Page récupérée avec succès !
------------------------------
Tableau trouvé avec 182 gares !
                               Gare        Département No dpt  \
0  Aéroport Charles-de-Gaulle 2 TGV  Seine-Saint-Denis     93   
1                              Agde            Hérault     34   
2                              Agen     Lot-et-Garonne     47   
3                    Aime-La Plagne             Savoie     73   
4               Aix-en-Provence TGV   Bouches-du-Rhône     13   

                       Région  \
0               Île-de-France   
1                   Occitanie   
2          Nouvelle-Aquitaine   
3        Auvergne-Rhône-Alpes   
4  Provence-Alpes-Côte d'Azur   

                                       Type de train  \
0                         TGV inOui, Ouigo, Eurostar   
1                                   TGV inOui, Ouigo   
2                                   TGV inOui, Ouigo   
3  TGV inOui (saisonnier), Ouigo (saisonnier), Eu...   
4  TGV inOui, TGV Lyria (saisonnier), 

/Users/yassermouaqqat/Desktop/ENSAE/2A/S1/Python pour la data science/Projet-Python-2A-ENSAE/.venv/lib/python3.12/site-packages/urllib3/connectionpool.py:1097: InsecureRequestWarning: Unverified HTTPS request is being made to host 'fr.wikipedia.org'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(
